## original code is =>
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

## Import Required Libraries

In [1]:
import torch
import torch.nn as nn

import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

## Import Visdom

아나콘다 터미널에서 `python -m visdom.server` 먼저 실행시키기

In [2]:
import visdom

vis = visdom.Visdom()
vis.close(env="main")

Setting up a new session...


''

## Define Value tracker

In [3]:
# loss와 Value 같이 표현
def value_tracker(value_plot, value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=value,
             win = value_plot,
             update='append'
             )

## GPU 사용 가능 여부 및 Random Seed 설정

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

## transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))

### How to Calculate mean and std in Normalize 

In [5]:
# transform을 구성
transform = transforms.Compose([
    transforms.ToTensor()
])

# Training data set 가져오기 
trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)

# CIFAR10 : train_data -> data
# Training data set 평균 구하기
train_data_mean = trainset.data.mean( axis=(0,1,2) )
# Training data set 표준편차 구하기
train_data_std = trainset.data.std( axis=(0,1,2) )


print(train_data_mean)
print(train_data_std)

# Max 값으로 나누어주기
train_data_mean = train_data_mean / 255
train_data_std = train_data_std / 255

# 평균 및 표준편차 출력
print('train_data_mean : ', train_data_mean)
print('train_data_std', train_data_std)


Files already downloaded and verified
[125.30691805 122.95039414 113.86538318]
[62.99321928 62.08870764 66.70489964]
train_data_mean :  [0.49139968 0.48215841 0.44653091]
train_data_std [0.24703223 0.24348513 0.26158784]


In [6]:
# train transform을 구성
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # padding을 4로 주고 랜덤하게 32로 뜯어 오겠다.
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)  # 일반적으로는 normalized를 하는 것이 좋다.
])

# test transform을 구성 -> train과 같은 분포를 가질 것이라 가정
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_data_mean, train_data_std)
])

# Training data set 가져오기
trainset = torchvision.datasets.CIFAR10(root='./cifar10', train=True,
                                        download=True, transform=transform_train)

# Train dataloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=0)

# Test data set 가져오기
testset = torchvision.datasets.CIFAR10(root='./cifar10', train=False,
                                       download=True, transform=transform_test)

# Test dataloader
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Make ResNet50 using resnet.py

In [7]:
import resnet
# import torchvision.models.resnet as resnet

In [8]:
# ResNet class 밖에 있는 것들 선언 
conv1x1=resnet.conv1x1
Bottleneck = resnet.Bottleneck
BasicBlock= resnet.BasicBlock

In [9]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 16      # CIFAR10은 처음에 많은 channel이 필요없다! size가 작기 때문
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,   # input size가 작아졌으므로 모두 알맞게 조정해준다.
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        
        #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # size가 작아서 maxpooling이 필요가 없다.
        
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        # input.shape = [batch,3,32,32]
        x = self.conv1(x)
        #x.shape =[1, 16, 32,32]
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)

        x = self.layer1(x)
        #x.shape =[1, 128, 32,32]
        x = self.layer2(x)
        #x.shape =[1, 256, 32,32]
        x = self.layer3(x)
        #x.shape =[1, 512, 16,16]
        x = self.layer4(x)
        #x.shape =[1, 1024, 8,8]
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [10]:
resnet50 = ResNet(resnet.Bottleneck, [3, 4, 6, 3], 10, True).to(device) 
#1(conv1) + 9(layer1) + 12(layer2) + 18(layer3) + 9(layer4) +1(fc)= ResNet50

In [11]:
resnet50

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

### Model Test

In [12]:
a=torch.Tensor(1,3,32,32).to(device)  # random input
out = resnet50(a)
print(out)

tensor([[ 0.0332, -0.0374,  0.0291, -0.0121, -0.0018,  0.0375,  0.0070,  0.0131,
          0.0420,  0.0400]], device='cuda:0', grad_fn=<AddmmBackward>)


In [13]:
# Loss function
criterion = nn.CrossEntropyLoss().to(device)

# Optimizer
optimizer = torch.optim.SGD(resnet50.parameters(), lr = 0.1, momentum = 0.9, weight_decay=5e-4)

# learning rate schedular
lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## Make plot

In [14]:
# Loss를 plot
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

# Accuracy를 plot
acc_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='Accuracy', legend=['Acc'], showlegend=True))

## Define acc_check function

In [15]:
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    
    # 중간에 accuarcy를 check할 때는 gradient update를 하면 안됨.
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    
    # Model을 중간중간 저장할 수 있게 해준다.
    if save:
        torch.save(net.state_dict(), "./model_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

## Training with (acc check + model save)

In [16]:
print(len(trainloader))
epochs = 5                   # epochs 조정

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    # learning rate update
    lr_sche.step()
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward 
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # backward
        loss.backward()
        # optimize
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(trainloader) ]))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    
    #Check Accuracy
    acc = acc_check(resnet50, testloader, epoch, save=1)
    value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    

print('Finished Training')

782
[1,    30] loss: 2.314
[1,    60] loss: 2.027
[1,    90] loss: 1.965
[1,   120] loss: 1.858
[1,   150] loss: 1.826
[1,   180] loss: 1.812
[1,   210] loss: 1.758
[1,   240] loss: 1.706
[1,   270] loss: 1.696
[1,   300] loss: 1.697
[1,   330] loss: 1.647
[1,   360] loss: 1.703
[1,   390] loss: 1.583
[1,   420] loss: 1.542
[1,   450] loss: 1.558
[1,   480] loss: 1.513
[1,   510] loss: 1.483
[1,   540] loss: 1.463
[1,   570] loss: 1.437
[1,   600] loss: 1.478
[1,   630] loss: 1.415
[1,   660] loss: 1.337
[1,   690] loss: 1.332
[1,   720] loss: 1.299
[1,   750] loss: 1.317
[1,   780] loss: 1.307
Accuracy of the network on the 10000 test images: 44 %
[2,    30] loss: 1.303
[2,    60] loss: 1.282
[2,    90] loss: 1.215
[2,   120] loss: 1.243
[2,   150] loss: 1.247
[2,   180] loss: 1.189
[2,   210] loss: 1.177
[2,   240] loss: 1.160
[2,   270] loss: 1.128
[2,   300] loss: 1.180
[2,   330] loss: 1.132
[2,   360] loss: 1.133
[2,   390] loss: 1.093
[2,   420] loss: 1.110
[2,   450] loss: 1.12

## Model Accuracy Testing

In [17]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = resnet50(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 67 %
